In [1]:
# Playing with expenses

########################################
#         Data Import from Sheets      #
########################################
import gspread
import pandas as pd
from datetime import datetime, timedelta
import calendar
import matplotlib.pyplot as plt
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.message import EmailMessage
import os
import mimetypes
from pathlib import Path
import json
from dotenv import load_dotenv

load_dotenv()

email_username = os.getenv("EMAIL_USERNAME")
email_password = os.getenv("EMAIL_PASSWORD")

# Set up credentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv('TRY'), scope)
json_keyfile_dict = json.loads(os.getenv("JSON_KEYFILE_DICT"))
creds = ServiceAccountCredentials.from_json_keyfile_dict(json_keyfile_dict, scope)
client = gspread.authorize(creds)


# Open Spreadsheet based on sheet key
spreadsheet_key = os.getenv("SPREADSHEET_KEY")
spreadsheet = client.open_by_key(spreadsheet_key)

## Open Expenses Sheet
sheet_title = 'Expenses'  
sheet = spreadsheet.worksheet(sheet_title)

# Extract all of the data
expenses_data = sheet.get_all_records()

# Convert from JSON to pandas Data Frame
expenses_data = pd.DataFrame(expenses_data)

# Make Timestamp a column that can be worked with Pandas library
expenses_data['Timestamp'] = pd.to_datetime(expenses_data['Timestamp'])

## Open Income Sheet
sheet_title = 'Income'  
sheet = spreadsheet.worksheet(sheet_title)

# Extract all of the data
income_data = sheet.get_all_records()

# Convert from JSON to pandas Data Frame
income_data = pd.DataFrame(income_data)

# Make Timestamp a column that can be worked with Pandas library
income_data['Timestamp'] = pd.to_datetime(income_data['Timestamp'])



C:\Users\lohan.rossi\AppData\Local\Temp\ipykernel_15680\2015754551.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
df = expenses_data

df['Year-Month'] = df['Timestamp'].dt.to_period('M')

# Group by 'Year-Month' and sum the 'Expense Amount'
monthly_expenses = df.groupby('Year-Month')['Expense Amount'].sum().reset_index()

# Display the result
print(monthly_expenses)
print(monthly_expenses['Expense Amount'].mean())


   Year-Month  Expense Amount
0     2023-03          307.72
1     2023-04          216.91
2     2023-05          137.87
3     2023-06          276.70
4     2023-07          272.17
5     2023-08          274.07
6     2023-09          213.10
7     2023-10          237.61
8     2023-11          273.05
9     2023-12          167.33
10    2024-01          265.38
11    2024-02          161.62
12    2024-03          119.26
13    2024-04          190.31
14    2024-05          396.53
15    2024-06          118.96
16    2024-07           27.13
17    2024-08          138.44
18    2024-09          452.28
19    2024-10          334.11
20    2024-11          491.56
21    2024-12          122.48
22    2025-01          667.90
23    2025-02          372.20
24    2025-03          514.67
25    2025-04          336.36
272.5276923076923


C:\Users\lohan.rossi\AppData\Local\Temp\ipykernel_15680\1798136331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year-Month'] = df['Timestamp'].dt.to_period('M')


In [3]:
df = income_data

df['Year-Month'] = df['Timestamp'].dt.to_period('M')

# Group by 'Year-Month' and sum the 'Expense Amount'
monthly_income = df.groupby('Year-Month')['Income Amount'].sum().reset_index()

# Display the result
print(monthly_income)

   Year-Month  Income Amount
0     2023-02          28.92
1     2023-03        5947.41
2     2023-04        3795.88
3     2023-05        3795.88
4     2023-06        3795.88
5     2023-07        3922.79
6     2023-08        3795.88
7     2023-09        5693.82
8     2023-10        3846.64
9     2023-11        3860.38
10    2023-12        3795.88
11    2024-01        4988.76
12    2024-02        3795.88
13    2024-03       10491.09
14    2024-04        5286.49
15    2024-05        4059.50
16    2024-06        4059.49
17    2024-07        4059.48
18    2024-08        9638.17
19    2024-09        4131.29
20    2024-10        3956.14
21    2024-11        3956.13
22    2024-12        3956.14
23    2025-01        5999.45
24    2025-02        3954.50
25    2025-03       15634.93
26    2025-04        3392.95


In [4]:
monthly_expense_vs_income = pd.merge(monthly_income, monthly_expenses, on='Year-Month')
monthly_expense_vs_income['Delta'] = monthly_expense_vs_income['Income Amount'] - monthly_expense_vs_income['Expense Amount']
monthly_expense_vs_income

,Year-Month,Income Amount,Expense Amount,Delta
0,2023-02,28.92,42.86,-13.94
1,2023-03,5947.41,2176.60,3770.81
2,2023-04,3795.88,2104.54,1691.34
3,2023-05,3795.88,1852.65,1943.23
4,2023-06,3795.88,2082.58,1713.30
5,2023-07,3922.79,5098.66,-1175.87
6,2023-08,3795.88,2814.46,981.42
7,2023-09,5693.82,2039.43,3654.39
8,2023-10,3846.64,1781.84,2064.80
9,2023-11,3860.38,7962.66,-4102.28


In [5]:
monthly_expense_vs_income['Delta'].cumsum()

0       -13.94
1      3756.87
2      5448.21
3      7391.44
4      9104.74
5      7928.87
6      8910.29
7     12564.68
8     14629.48
9     10527.20
10    12043.88
11    14894.89
12    17275.95
13    25562.64
14    28681.35
15    30563.69
16    30021.22
17    31841.35
18    30761.88
19    30740.73
20    32199.24
21    32145.45
22    32469.05
23    33910.64
24    34608.93
25    41538.12
26    42402.35
Name: Delta, dtype: float64

In [6]:
monthly_expense_vs_income['Delta'].median()

1516.6800000000003

In [8]:
# My budget is basically $4,000

rent = 500
insurance = 71 # 2 cars
gas = 100
charity = 250
health = 35
phone = 33.75
internet = 60

monthly_expenses_deal = rent + insurance + gas + charity + health + phone + internet
monthly_expenses_deal

1049.75